##Call the API with my variables for my research

> List of some states "46 SD", "47 TN","10 DE", "2 AK","5 AR","24 MD","31 NE"

CBSA:

"46 SD",
* "43620 Sioux Falls, SD",

"47 TN"
* "16860 Chattanooga, TN-GA",
* "28940 Knoxville, TN",
* "34980": "Nashville-Davidson--Murfreesboro--Franklin, TN",
* "28700": "Kingsport-Bristol-Bristol, TN-VA",
* "32820": "Memphis, TN-MS-AR",
* "17300": "Clarksville, TN-KY",
* "27740": "Johnson City, TN",
* "17420": "Cleveland, TN",

"10 DE"
* "20100 Dover, DE",
* "428": "Philadelphia-Reading-Camden, PA-NJ-DE-MD",
* "41540": "Salisbury, MD-DE",
*
* 0 city for "2 AK"

"5 AR",
* "22900 Fort Smith, AR-OK",
* "30780 Little Rock-North Little Rock-Conway, AR",
* "22220 Fayetteville-Springdale-Rogers, AR-MO",
* "38220 Pine Bluff, AR",
* "32820 Memphis, TN-MS-AR",
*
"24 MD",
* "25180": "Hagerstown-Martinsburg, MD-WV",
* "428": "Philadelphia-Reading-Camden, PA-NJ-DE-MD",
* "12580": "Baltimore-Columbia-Towson, MD",
* "37980": "Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",
* "15680": "California-Lexington Park, MD",
* "41540": "Salisbury, MD-DE",

##CONSTANTES

In [1]:
# (10) IMPORT LIBRARY
import requests
import pandas as pd
import regex as re
import time
from datetime import datetime as dt
import json
import logging

In [2]:
# (20) CONSTANTES

#Dict of all states
all_states = requests.get('https://api.census.gov/data/2024/cps/basic/jan/variables.json')
states = json.loads(all_states.text)
states_dict = states['variables']['STATE']['values']['item']
states = [state for state in states_dict.keys()]

#All year of the study


##Calling the API (1st step)

In [52]:
# (30) CLASS API and Decorator [log_activity2]

def log_activity2(func):
    def wrapper(*args, **kwargs):
        # Create a logger
        logger = logging.getLogger('activity_logger')
        logger.setLevel(logging.INFO)

        # Create a file handler and set the logging format
        file_handler = logging.FileHandler('activity.log')
        formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
        file_handler.setFormatter(formatter)

        # Add the file handler to the logger
        logger.addHandler(file_handler)

        # Log the activity before calling the wrapped function
        logger.info(f'Activity: {func.__name__}')

        # Call the wrapped function
        return func(*args, **kwargs)

    return wrapper


class API:
  """This generic class API will handle at once all request to the CPS api
  How to?:
  > instanciate a variable at once, then use it to browse through all variables (not data) on the Census API json file
  > example api:    api=API()

  Properties:
  > allVars_dict (variable): a dict of all variables for each 'year' (key)
  > examples:
    - api.allVars_dict[2023]        * to se all the variables (dict) in 2023
    - 'GTCBSA' in api.allVars_dict[2024].keys() *                is <GTCBSA> in 2024?
    - api.allState_dict[2021]       * all stateID:description in year 2021"""

  def __init__(self) -> None:
    self.get()

  @log_activity2
  def get(self):
    self.allVars_dict = {year_: year_vars for year_, year_vars in \
            zip(self.allYearList(), [json.loads(requests.get('https://api.census.gov/data/'+ str(year_) +'/cps/basic/jan/variables.json').text)['variables'] for year_ in self.allYearList()])}

  def store(self):
    pass

  @log_activity2
  def allStatesIdList(self, year:int=2020):
    return list(self.allVars_dict[year]['GESTFIPS']['values']['item'].keys())

  @log_activity2
  def allMonthList3TLA(self):
    month_dict = {1:"jan",2:"feb",3:"mar",4:"apr",5:"may",6:"jun",7:"jul",8:"aug",9: "sep",10: "oct",11:"nov",12:"dec"}
    #return next(key for key, value in month_dict.items() if value == m)
    return list(month_dict.values())

  def allYearList(self):
    return range(2010, 1+dt.now().year)

  @log_activity2
  def getVarsNameFromUrl(self, url):
    return re.findall(r'([A-Z]+)', url)


api=API()

INFO:activity_logger:Activity: get


In [53]:
# (40) CLASS MIXCOMP
#Decorator to track API response time
def track_response_time(func):
    def wrapper(self, *args, **kwargs):
        start_time = time.time()
        result = func(self, *args, **kwargs)
        end_time = time.time()
        response_time = end_time - start_time
        self.response_time = response_time  # Store in the class
        print(f"Function: '{func.__name__}()'. Census API query took: {response_time} seconds")
        return result
    return wrapper


#Class for unit-request (specific State, specific Year, specific Month)
class MixComp:
  """
This class will pull the Mix composition about the US population from one state.
Code	    Label
tabulate  Counts of instances
PEMNTVTY	Demographics-native country of mother
PEMARITL	Demographics-marital status
PESEX	    Demographics-sex
PRCITSHP	Demographics-United States citizenship group
GTCBSA	  Demographics-city level
STATE	    FIPS STATE Code
GTCBSA	  Demographics-city level

  """
  def __init__(self, stateID, year, month):
      key = '&key='+'804d0a1a18d1de70764950c78e2a3a42d3d45e48'
      self.url = 'https://api.census.gov/data/'+ str(year) +'/cps/basic/'+ month + '?tabulate=weight(PWSSWGT)&row+for&row+PEMNTVTY&row+PEMARITL&row+PESEX&row+PRCITSHP&row+GTCBSA&for=state:' + str(stateID) + key
      self.stateID = stateID
      self.year = year
      self.month = month
      self.response_time = 0 # how long the API took to run on Census CPS
      self.dataAPI = None
      self.dataText = None
      self.data4Df = []
      self.df = pd.DataFrame(None) #Will contain the final df(dataframe)
      self.initLogger()

  def initLogger(self):
      # Create a logger
      self.logger = logging.getLogger('activity_logger')
      self.logger.setLevel(logging.INFO)
      # Create a file handler and set the logging format
      file_handler = logging.FileHandler('activity.log')
      formatter = logging.Formatter('%(asctime)s - %(levelname)s - [w86] %(message)s')
      file_handler.setFormatter(formatter)
      # Add the file handler to the logger
      self.logger.addHandler(file_handler)

  def __str__(self):
      return f"""
      Mix Composition of the US population: stateID:={self.stateID}, month={self.month}, year={self.year}
      Dataframe head:
         {self.df.head(3)}"""

  def __repr__(self):
      return f"MixComp({self.stateID}, {self.year}, {self.month})"

  def __len__(self):
      return self.df.shape[0]

  def __add__(self, other):
      return pd.concat([self.df, other.df], axis=0)

  # to convert in class
  def checkVariable(self, year, url):
      """
      this function will check if a set of variables are declared on the Census API. Using the an url as input. Vars are in ALL_CAPS
      """
      api=API()
      api.get()
      matches = api.getVarsNameFromUrl(url)
      cps_vars = api.allVars_dict[year].keys()
      res = dict()
      for var in matches:
        res[var] = True if var in cps_vars else False
      res['status'] = all([status for status in res.values()])
      return res

  @track_response_time
  def get(self):

      #This section will validate all variables and replace the city's with the right one if year>=2024
#      check = self.checkVariable(self.year, self.url)
#      print (f"checkVariable() = {check}")
#      if not check['status']:
#        print(f"BEFORE - Variables = {check}")
#        if (self.year>=2024) and ('CBSA' not in check.keys()): #before 2024, City variable in 'GTCBSA', starting on 2024, City variable was 'CBSA'
#          self.url = re.sub('GTCBSA', 'CBSA', self.url)
#          print(f"AFTER -  Variables = {self.checkVariable(self.year, self.url)}\nNew URL: {self.url}")
#        else:
#          raise(f"Please check all your Census variables. Unable to find some in the API call. Variables = {check}")

      try:
        # download the Demographics_Native_Country_Of_Mother for Mix-Compostion
        self.dataAPI = requests.get(self.url)
        return dict({'response time in secs': round(self.response_time,0), 'status': True if 200==self.dataAPI.status_code else False, 'code':self.dataAPI.status_code, 'comment': 'url:' + self.url})
        None
      except requests.exceptions.RequestException as e:
          # Handle connection errors or HTTP errors
          raise("STOP ERROR: Tools for Data Science - Semester Project: An error occurred:\n [[[[", e, "]]]]")

      except AttributeError:
          raise("STOP ERROR: Please check your variable name!")

  def cleaning(self):
      self.dataText = self.dataAPI.text
      pattern_cleaning = r'\[0,.*?\],?\n?' #remove the data with no tabulate information (tabulate=0)
      self.dataText = re.sub(pattern_cleaning, '', self.dataText)
      pattern_cleaning = r'\n' #remove retunr line
      self.dataText = re.sub(pattern_cleaning, '', self.dataText)
      self.logger.info(f'> Cleaning dataAPI but removing data with no tabulate information (tabulate=0)')

  def preping(self):
      ##Separe cols from data (numbers)
      pattern = r'.*?(\["tabulate".*?"GTCBSA"\]),([\s\S]*)'
      matches = re.search(pattern, self.dataText)
      data_cols = matches.group(1)
      data_vals = matches.group(2)[:-1] #removing the last char "]" at the end.

      self.data_cols = re.findall(r'"(.*?)"', data_cols) #Transform string_column in List_column

      #Transform string_row in List of Lists then Matrix
      rows = re.findall(r'\[(.*?)\]', data_vals)
      for row in rows:
        row_list = re.findall(r'-?\d+(?:\.\d+)?', row)
        self.data4Df.append([int(elt) for elt in row_list])

      self.logger.info(f'> len(self.data4Df): {len(self.data4Df)}')

  def buildingDf(self):
      """
      ##Definition of variables:
      * tabulate: represents the information related to population counts.
      * state: column provides information about the state
      1. PEMNTVTY Demographics-native country of mother
      2. PEMARITL Demographics-marital status
      3. PESEX Demographics-sex
      4. PRCITSHP Demographics-United States citizenship group
      5. GTCBSA Demographics-city level
      We will understand the population composition by state about how mixed it is based on the native country of mothers,
      marital status distribution, gender demographics, citizenship status, at the city-level characteristics.
      This data can be used to uncover trends, patterns, and insights into the social and cultural fabric of the US population.
      """
      ##Convert the result to the Df **mix_composition**
      self.df = pd.DataFrame(self.data4Df, columns=self.data_cols)

      ##Building the YYYYMM column
      self.df['YYYYMM'] = pd.Series([dt.strptime(str(self.year)+ '-' +str(self.month), "%Y-%b") for _ in range(self.df.shape[0])])

      self.logger.info(f'> Built Dataframe for stateiD:{self.stateID}, YYYYMM:{str(self.year)}-{str(self.month)}')

  def save(self, path=''):
      try:
        if path == '':
          stateID = '0'+ str(self.stateID) if len(self.stateID)<2 else str(self.stateID) #fixing stateID in 2 digits
          path = str(self.year)+'-'+ str(self.month)+'-'+ str(stateID)+'-MixComp.csv'
        r = self.df.to_csv( path, index=False)
        self.logger.info(f'> Created CSV file: {path}')
        return r
      except FileNotFoundError:
        raise (f'Check the path {path} of the file')

  def build(self):
      self.logger.info(f'> Called main(self) method - Entrance')
      self.logger.info(f'> Input Variables = self.url: {self.url}, self.stateID: {self.stateID}, self.year: {self.year}, self.month: {self.month}')

      self.logger.info(f'> Called main(self) - get() method - Will take some time usually 3-5min')
      res = self.get()
      self.logger.info(f'Called main(self) - line: get() method - Result:{res}')

      self.cleaning()
      self.preping()
      self.buildingDf(); self.logger.info(f'> Called main(self) - buildingDf() method')

      self.logger.info(f'> Called main(self) - save() method')
      self.save()

      self.logger.info(f'> Called main(self) method - End')


In [50]:
# (60) Going through All Months at once
for year in api.allYearList():
  for stateID in api.allStatesIdList(): #['31', '47']:
    for month in api.allMonthList3TLA(): #['jan', 'feb']:
      dataA = MixComp(stateID=stateID, year=year, month=month)
      dataA.build()

INFO:activity_logger:> Called main(self) method - Entrance
INFO:activity_logger:> Input Variables = self.url: https://api.census.gov/data/2010/cps/basic/jan?tabulate=weight(PWSSWGT)&row+for&row+PEMNTVTY&row+PEMARITL&row+PESEX&row+PRCITSHP&row+GTCBSA&for=state:31&key=804d0a1a18d1de70764950c78e2a3a42d3d45e48, self.stateID: 31, self.year: 2010, self.month: jan
INFO:activity_logger:> Called main(self) - get() method - Will take some time usually 3-5min
INFO:activity_logger:Called main(self) - line: get() method - Result:{'response time in secs': 0, 'status': True, 'code': 200, 'comment': 'url:https://api.census.gov/data/2010/cps/basic/jan?tabulate=weight(PWSSWGT)&row+for&row+PEMNTVTY&row+PEMARITL&row+PESEX&row+PRCITSHP&row+GTCBSA&for=state:31&key=804d0a1a18d1de70764950c78e2a3a42d3d45e48'}


Function: 'get()'. Census API query took: 176.77804517745972 seconds


INFO:activity_logger:> Cleaning dataAPI but removing data with no tabulate information (tabulate=0)
INFO:activity_logger:> len(self.data4Df): 195
INFO:activity_logger:> Built Dataframe for stateiD:31, YYYYMM:2010-jan
INFO:activity_logger:> Called main(self) - buildingDf() method
INFO:activity_logger:> Called main(self) - save() method


TypeError: exceptions must derive from BaseException

In [6]:
# (50) TEST
stateID = states[1] #47 #range(1-56), without 3, 7, 14, 43
year = 2023 # range(2010-2023)
month = month_dict[10] #range(1-12)
#dataA = MixComp(stateID, year, month)
#dataA.build()

stateID = states[2] #range(1-56), without 3, 7, 14, 43
year = 2015 # range(2010-2023)
month = month_dict[2] #range(1-12)
#dataB = MixComp(stateID, year, month)
#dataB.build()

#dataR = dataA + dataB
#dataR

INFO:activity_logger:> Called main(self) method - Entrance
INFO:activity_logger:> Called main(self) - get() method - Will take some time usually 3-5min
INFO:activity_logger:Called main(self) - line: get() method - Result:{'response time in secs': 0, 'status': True, 'code': 200, 'comment': 'url:https://api.census.gov/data/2023/cps/basic/oct?tabulate=weight(PWSSWGT)&row+for&row+PEMNTVTY&row+PEMARITL&row+PESEX&row+PRCITSHP&row+GTCBSA&for=state:47&key=804d0a1a18d1de70764950c78e2a3a42d3d45e48'}


Function: 'get()'. Census API query took: 184.5014727115631 seconds


INFO:activity_logger:> Cleaning dataAPI but removing data with no tabulate information (tabulate=0)
INFO:activity_logger:> len(self.data4Df): 247
INFO:activity_logger:> Called main(self) - buildingDf() method
INFO:activity_logger:> Built Dataframe for stateiD:47, YYYYMM:2023-oct
INFO:activity_logger:> Called main(self) - save() method - Beginning
INFO:activity_logger:> Created CSV file: 2023-oct-MixComp.csv
INFO:activity_logger:> Called main(self) - save() method - End
INFO:activity_logger:> Called main(self) method - End
INFO:activity_logger:> Called main(self) method - Entrance
INFO:activity_logger:> Called main(self) - get() method - Will take some time usually 3-5min
INFO:activity_logger:Called main(self) - line: get() method - Result:{'response time in secs': 0, 'status': True, 'code': 200, 'comment': 'url:https://api.census.gov/data/2015/cps/basic/feb?tabulate=weight(PWSSWGT)&row+for&row+PEMNTVTY&row+PEMARITL&row+PESEX&row+PRCITSHP&row+GTCBSA&for=state:10&key=804d0a1a18d1de707649

Function: 'get()'. Census API query took: 221.0948247909546 seconds


INFO:activity_logger:> Cleaning dataAPI but removing data with no tabulate information (tabulate=0)
INFO:activity_logger:> len(self.data4Df): 206
INFO:activity_logger:> Called main(self) - buildingDf() method
INFO:activity_logger:> Built Dataframe for stateiD:10, YYYYMM:2015-feb
INFO:activity_logger:> Called main(self) - save() method - Beginning
INFO:activity_logger:> Created CSV file: 2015-feb-MixComp.csv
INFO:activity_logger:> Called main(self) - save() method - End
INFO:activity_logger:> Called main(self) method - End


##Prep'ing the _columns_ and _data values_ for the _df_ called **mix_composition**

##Convert fto df

#class Creation
Now we have all we need for our analysis, let's try to reorganize what we build so far into a class called _MixComp_

##Let's test the Class MixComp